# Проект защиты данных клиентов страховой компании

Цель проекта - защитить данные клиентов страховой компании. Разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию, а качество моделей машинного обучения при этом не ухудшилось. Докажем корректность работы этого алгоритма.

## 1. Загрузка данных

In [15]:
import pandas as pd
from sklearn.datasets import make_spd_matrix
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from numpy.linalg import inv, cond
import numpy as np

In [16]:
insurance_data = pd.read_csv('/datasets/insurance.csv')

In [17]:
print(insurance_data.info())
insurance_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Предоставленные данные состоят из 5 столбцов. 4 из них - признаки, а один - `Страховые выплаты` - целевой признак. Пропущенных значений нет, но можно проверить датасет на наличие дублирующихся наблюдений.

In [18]:
insurance_data.duplicated().value_counts()

False    4847
True      153
dtype: int64

Дубликаты есть. Так как совпадение по всем признакам крайне маловероятно, избавимся от них.

In [19]:
insurance_data.drop_duplicates(inplace=True)

In [20]:
insurance_data.duplicated().value_counts()

False    4847
dtype: int64

Переименуем столбцы датасета для удобства.

In [21]:
insurance_data.columns = ['sex', 'age', 'salary', 'family_members', 'insurance_payments']

## 2. Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

- $E$  — единичная матрица


Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

Докажем, что:

$$
a = Xw = X'w' = XPw' = a'
$$

Чтобы это равенство выполнялось, необходимо соблюсти условие:

$$
w' = P^{-1} w
$$

Получим следующее равенство:

$$
a = Xw = X'(P^{-1} 𝑤) = XP(P^{-1} 𝑤) = XEw = Xw = a
$$

**Ответ:** Нет, не изменится.

**Обоснование:** Как известно, обратная для квадратной матрицы P — матрица P с верхним индексом -1, произведение которой на P равно единичной матрице. Если любую матрицу умножить на единичную (или наоборот), получится эта же матрица. Поэтому качество линейной регрессии не изменится - матрицы будут одинаковы.

## 3. Алгоритм преобразования

**Алгоритм**

Создадим случайную обратимую матрицу P, затем умножим матрицу с признаками X на матрицу P.

$$
𝑋@𝑃=𝑍
$$

Для возврата к исходной матрице X, умножим новую матрицу Z на обратную матрицу P^-1.

$$
𝑍@P^{-1}=𝑋
$$

**Обоснование**

Как было доказано в шаге 2, качество линейной регрессии не изменится при умножении матрицы признаков на любую обратимую матрицу P. Поэтому можно следовать описанному выше алгоритму без опасения получить другой результат линейной регрессии.

## 4. Проверка алгоритма

Выделим из датасета набор признаков и целевой признак.

In [22]:
insurance_features = insurance_data.drop('insurance_payments', axis=1)

In [23]:
insurance_target = insurance_data['insurance_payments']

Получим матрицу из значений набора признаков.

In [24]:
insurance_features_matrix = insurance_features.values

In [25]:
insurance_features_matrix 

array([[1.00e+00, 4.10e+01, 4.96e+04, 1.00e+00],
       [0.00e+00, 4.60e+01, 3.80e+04, 1.00e+00],
       [0.00e+00, 2.90e+01, 2.10e+04, 0.00e+00],
       ...,
       [0.00e+00, 2.00e+01, 3.39e+04, 2.00e+00],
       [1.00e+00, 2.20e+01, 3.27e+04, 3.00e+00],
       [1.00e+00, 2.80e+01, 4.06e+04, 1.00e+00]])

Создадим рандомную матрицу нужного размера с помощью инструмента `make_spd_matrix` библиотеки `sklearn`.

In [26]:
random_matrix = make_spd_matrix(n_dim=4, random_state=12345)
display(random_matrix)

array([[ 1.37245706, -1.03845957, -0.84389737, -0.26033015],
       [-1.03845957,  2.87886199,  1.67157893,  0.48470484],
       [-0.84389737,  1.67157893,  2.10204907,  0.3257384 ],
       [-0.26033015,  0.48470484,  0.3257384 ,  1.01695329]])

Проверим матрицу на обратимость - вероятность сгенерировать необратимую матрицу очень мала, но все же есть. При использовании необратимой матрицы дальнейшая работа по описанному алгоритму невозможна. Для проверки будем использовать функцию `numpy.linalg.inv()`, если матрица необратима, то код упадет с ошибкой.

In [27]:
inv(random_matrix)

array([[ 1.06341364,  0.23949912,  0.22304514,  0.08662924],
       [ 0.23949912,  0.72198481, -0.45683306, -0.13647886],
       [ 0.22304514, -0.45683306,  0.93223396, -0.0237669 ],
       [ 0.08662924, -0.13647886, -0.0237669 ,  1.07816747]])

Матрица обратима, двигаемся дальше.

Перемножим матрицы.

In [28]:
transformed_matrix = insurance_features_matrix @ random_matrix
features_transformed = list(transformed_matrix)

Разделим набор признаков и целевой признак исходного датасета на обучающую и тестовую выборки.

In [29]:
features_train_origin, features_test_origin = train_test_split(insurance_features, test_size=0.25, random_state=12345)

In [30]:
target_train_origin, target_test_origin = train_test_split(insurance_target, test_size=0.25, random_state=12345)

Создадим модели.

In [31]:
model1 = LinearRegression()
model2 = LinearRegression()

Обучим первую модель, сделаем предсказание и оценим качество модели.

In [32]:
model1.fit(features_train_origin, target_train_origin)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [33]:
predicted_origin = model1.predict(features_test_origin)

In [34]:
print("R2 исходного набора данных:", r2_score(predicted_origin, target_test_origin))

R2 исходного набора данных: -0.26966120247765524


Разделим набор признаков преобразованного датасета на обучающую и тестовую выборки. Обучим вторую модель, сделаем предсказание и оценим качество модели.

In [35]:
features_train_transformed, features_test_transformed = train_test_split(features_transformed, test_size=0.25, random_state=12345)

In [36]:
model2.fit(features_train_transformed, target_train_origin)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [37]:
predicted_transformed = model2.predict(features_test_transformed)

In [38]:
print("R2 преобразованного набора данных:", r2_score(predicted_transformed, target_test_origin))

R2 преобразованного набора данных: -0.26966120247736036


Показатели R2 очень низкие, зато не отличаются (или отличаются незначительно).

Попробуем восстановить исходную матрицу, используя формулу из шага 3.

$$
𝑍@P^{-1}=𝑋
$$

In [39]:
final_matrix = transformed_matrix.dot(inv(random_matrix))

In [40]:
#сравним округленные матрицы
np.round(final_matrix) == np.round(insurance_features_matrix)

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       ...,
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

Матрицы одинаковы. Таким образом, мы реализовали алгоритм, описанный в шаге 3 и доказали, что качество линейной регрессии не меняется при умножении матрицы признаков на любую обратимую матрицу P. 